In [1]:
import sys
sys.path.insert(1, './env')
sys.path.insert(1, './agents')

import gym
import pylab
import random
import strategy_imitation, sarsa, ddqn, random_agent, a2c, model_based,graph_ai
import aa_gun,jet_table_simple
import numpy as np
from collections import deque
import keras
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#массовый тест моделей

In [ ]:
hard_mode = 0 #0 - ещё не, 1,2 - сейчас работает, 3 - уже всё
#Проверь на зенитке, на cartpole и на mountain car
EPISODES=1400

print('_____',pd.Timestamp.now())
#здесь весь код от инициализации модели до выдачи scores.
# In case of CartPole-v1, maximum length of episode is 500

env = jet_table_simple.jet_table_simple_env()
#env = gym.make('Seaquest-ramNoFrameskip-v0')
#env=CartPoleEnv9()
# get size of state and action from environment
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

#agent = sarsa.SarsaAgent(state_size, action_size)
goal_state_list = [[100, 0, 0],[0, 0.1, -0.1],[2000, 0, 1]]
agent = graph_ai.GraphAI(state_size, action_size, goal_state_list=goal_state_list)
agent.rl.train_start=340
#agent.train_start=7000
#agent.epsilon_decay=0.9999
agent.render=True

scores, episodes = [], []
reward_lst = []
s_list=[]
a_list=[]
agent.epsilon = 1

for e in range(EPISODES):
    done = False
    score = 0
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    

    while not done:
        if (e in range(0,18)) or (e in range(20,22)) or (e in range(30,32)) or (e in range(40,42)) or (e in range(50,52)) or (e in range(100,103)) or (e in range(200,202)) or (e in range(300,306)) or (e in range(400,406)) or (e in range(500,506)) or (e in range(600,604)):
            if agent.render:
                env.render()

        # get action for the current state and go one step in environment)
        action = agent.get_action(state)
        
        
        if (hard_mode==0) and (np.random.rand()<0.99):
            hard_mode=1
        if hard_mode==1:   
            if state[0][2]>0:#dy>1
                action=3
                #vy--
            else:
                hard_mode=2
        if hard_mode==2: 
            hard_mode=3
            if state[0][1]>10:#dx>1
                action=1
                #vx--
            else:
                hard_mode=3
        
        
        next_state, reward, done, info = env.step(action)
        next_state = np.reshape(next_state, [1, state_size])
        # if an action make the episode end, then gives penalty of -100
            

        # save the sample <s, a, r, s'> to the replay memory
        agent.append_sample(state, action, reward, next_state, done)
        #if next_state[0,11]!=reward:
        #    print('state[13]!=reward',state[0,11],reward)
        #
        s_list.append(state)
        a_list.append(action)
        reward_lst.append(reward)
        #

        # every time step do the training
        agent.train_model()
        score += reward
        state = next_state

        if done:
            # every episode update the target model to be same with model
            agent.update_target_model()

            # every episode, plot the play time
            scores.append(score)
            episodes.append(e)
            pylab.plot(episodes, scores, 'b')
            #pylab.savefig("./save_graph/aa_gun_dqn.png")
            try:
                print("episode:", e, "  score:", score,np.mean(scores), "  memory length:",
                      len(agent.rl.s), "  epsilon:", agent.epsilon)
            except Exception:
                print("episode:", e, "  score:", score,np.mean(scores), "  memory length:",
                      len(agent.rl.s), "  epsilon:", agent.epsilon)


_____ 2020-07-26 19:20:48.792346
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 300)               2100      
_________________________________________________________________
batch_normalization (BatchNo (None, 300)               1200      
_________________________________________________________________
dropout (Dropout)            (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 300)               90300     
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_2 (Dense)        

routing nan
learning nan
acting 0.0009001960784313725
node_adding nan
A0 0 days 00:00:00.016347
routing nan
learning nan
acting 0.0008980192307692308
node_adding nan
A0 0 days 00:00:00.016693
routing nan
learning nan
acting 0.0008954339622641509
node_adding nan
A0 0 days 00:00:00.016319
routing nan
learning nan
acting 0.0008938888888888889
node_adding nan
A0 0 days 00:00:00.016547
routing nan
learning nan
acting 0.0008923454545454545
node_adding nan
A0 0 days 00:00:00.016860
routing nan
learning nan
acting 0.0008951250000000001
node_adding nan
A0 0 days 00:00:00.015938
routing nan
learning nan
acting 0.0008895263157894737
node_adding nan
A0 0 days 00:00:00.016749
routing nan
learning nan
acting 0.0008901206896551725
node_adding nan
A0 0 days 00:00:00.016332
routing nan
learning nan
acting 0.0008885762711864408
node_adding nan
A0 0 days 00:00:00.016372
routing nan
learning nan
acting 0.0008856000000000002
node_adding nan
A0 0 days 00:00:00.016698
routing nan
learning nan
acting 0.000886

A0 0 days 00:00:00.015902
routing nan
learning nan
acting 0.0009392394366197183
node_adding nan
A0 0 days 00:00:00.015044
routing nan
learning nan
acting 0.0009379999999999998
node_adding nan
A0 0 days 00:00:00.016636
routing nan
learning nan
acting 0.0009380902777777778
node_adding nan
A0 0 days 00:00:00.016342
routing nan
learning nan
acting 0.0009375655172413794
node_adding nan
A0 0 days 00:00:00.017103
routing nan
learning nan
acting 0.000942178082191781
node_adding nan
A0 0 days 00:00:00.015815
routing nan
learning nan
acting 0.0009423537414965988
node_adding nan
A0 0 days 00:00:00.016451
routing nan
learning nan
acting 0.0009425878378378379
node_adding nan
A0 0 days 00:00:00.016328
routing nan
learning nan
acting 0.0009413221476510068
node_adding nan
A0 0 days 00:00:00.016989
routing nan
learning nan
acting 0.0009437733333333334
node_adding nan
A0 0 days 00:00:00.016292
routing nan
learning nan
acting 0.0009448145695364241
node_adding nan
A0 0 days 00:00:00.015967
routing nan
lea

routing nan
learning nan
acting 0.0009511724137931034
node_adding nan
A0 0 days 00:00:00.016560
routing nan
learning nan
acting 0.0009514248927038626
node_adding nan
A0 0 days 00:00:00.016433
routing nan
learning nan
acting 0.0009516282051282052
node_adding nan
A0 0 days 00:00:00.016559
routing nan
learning nan
acting 0.0009516127659574469
node_adding nan
A0 0 days 00:00:00.016158
routing nan
learning nan
acting 0.0009511440677966101
node_adding nan
A0 0 days 00:00:00.016623
routing nan
learning nan
acting 0.0009514388185654009
node_adding nan
A0 0 days 00:00:00.016417
routing nan
learning nan
acting 0.0009517268907563024
node_adding nan
A0 0 days 00:00:00.016477
routing nan
learning nan
acting 0.0009523556485355648
node_adding nan
A0 0 days 00:00:00.016426
routing nan
learning nan
acting 0.0009532583333333334
node_adding nan
A0 0 days 00:00:00.016217
routing nan
learning nan
acting 0.000953033195020747
node_adding nan
A0 0 days 00:00:00.016555
routing nan
learning nan
acting 0.0009535

routing nan
learning nan
acting 0.0009721950464396286
node_adding nan
A0 0 days 00:00:00.016447
routing nan
learning nan
acting 0.000971996913580247
node_adding nan
A0 0 days 00:00:00.016978
routing nan
learning nan
acting 0.0009726707692307693
node_adding nan
A0 0 days 00:00:00.015897
routing nan
learning nan
acting 0.0009719171779141105
node_adding nan
A0 0 days 00:00:00.016896
routing nan
learning nan
acting 0.0009728470948012234
node_adding nan
A0 0 days 00:00:00.015791
routing nan
learning nan
acting 0.0009721067073170731
node_adding nan
A0 0 days 00:00:00.016682
routing nan
learning nan
acting 0.0009718966565349543
node_adding nan
A0 0 days 00:00:00.016567
routing nan
learning nan
acting 0.0009716909090909091
node_adding nan
A0 0 days 00:00:00.016355
routing nan
learning nan
acting 0.0009714531722054381
node_adding nan
A0 0 days 00:00:00.016458
routing nan
learning nan
acting 0.0009712469879518073
node_adding nan
A0 0 days 00:00:00.016598
routing nan
learning nan
acting 0.0009714

...

1/1 [==============================] - 0s 1ms/step - loss: 2.0810
Epoch 37/60
1/1 [==============================] - 0s 1ms/step - loss: 3.0642
Epoch 38/60
1/1 [==============================] - 0s 1ms/step - loss: 2.9374
Epoch 39/60
1/1 [==============================] - 0s 1ms/step - loss: 3.0438
Epoch 40/60
1/1 [==============================] - 0s 1ms/step - loss: 3.4375
Epoch 41/60
1/1 [==============================] - 0s 2ms/step - loss: 4.0905
Epoch 42/60
1/1 [==============================] - 0s 1ms/step - loss: 2.3645
Epoch 43/60
1/1 [==============================] - 0s 1ms/step - loss: 2.8962
Epoch 44/60
1/1 [==============================] - 0s 1ms/step - loss: 3.1009
Epoch 45/60
1/1 [==============================] - 0s 1ms/step - loss: 2.4868
Epoch 46/60
1/1 [==============================] - 0s 1ms/step - loss: 3.4147
Epoch 47/60
1/1 [==============================] - 0s 1ms/step - loss: 2.2104
Epoch 48/60
1/1 [==============================] - 0s 1ms/step - loss: 1.928

go from [13.6        17.97500027] to [0. 1.], action vy++
GG 0 days 00:00:00.000100
routing 3.39072075
learning 1.8965592857142857
acting 0.039881213903743315
node_adding 0.2860065
A0 0 days 00:00:00.006011
go from [13.6        17.97500027] to [0. 1.], action vy++
GG 0 days 00:00:00.000100
routing 3.39072075
learning 1.8965592857142857
acting 0.039865135999999995
node_adding 0.2860065
A0 0 days 00:00:00.010767
go from [13.6        17.97500027] to [0. 1.], action vy++
edges rewrite success
GG 0 days 00:00:00.000111
routing 3.39072075
learning 1.8965592857142857
acting 0.03994873670212766
node_adding 0.2860065
A0 0 days 00:00:00.005809
go from [13.6        17.97500027] to [0. 1.], action vy++
GG 0 days 00:00:00.000099
routing 3.39072075
learning 1.8965592857142857
acting 0.03992763660477454
node_adding 0.2860065
A0 0 days 00:00:00.006089
GG 0 days 00:00:00.000109
routing 3.39072075
learning 1.8965592857142857
acting 0.039825306878306876
node_adding 0.2860065
A0 0 days 00:00:00.009581
go 

s1,s2 48.0,15.804600000511982,17.97500026847228 100,0,0
path ready
predicting distances 2020-07-26 19:21:58.783110
edges_coord_np [100.   0.   0.]
edges_values predicted: min max std avg 1655.9928 122116.3 47881.773 57568.156 len edges_values 8 2020-07-26 19:21:58.812770
nodes_count 11 edges_count 59
s1,s2 48.0,15.804600000511982,17.97500026847228 0,0.1,-0.1
cannot find path. s1 in graph: True s2 in graph: False
making new edges to s2, count_additional_edges 20 nodes 11
s1,s2 48.0,15.804600000511982,17.97500026847228 0,0.1,-0.1
path ready
predicting distances 2020-07-26 19:22:01.047836
edges_coord_np [100.   0.   0.]
edges_values predicted: min max std avg 80400.0 48723680.0 17921186.0 9110542.0 len edges_values 10 2020-07-26 19:22:01.076116
nodes_count 12 edges_count 80
s1,s2 48.0,15.804600000511982,17.97500026847228 2000,0,1
cannot find path. s1 in graph: True s2 in graph: False
making new edges to s2, count_additional_edges 20 nodes 12
s1,s2 48.0,15.804600000511982,17.97500026847228

ssr rmse 0.5573779365625718 ssr rmse/std 1.330569571821459 ssr rmse/mean 1.7351914561118675 s.shape (418, 3)
ssar 1.2481053714772645  rmse/std 2.1430116597697375 ssar rmse/mean 3.8855175902204984
routing 4.269481111111111
learning 3.361877
acting 0.09839016985645932
node_adding 0.2860065
A0 0 days 00:00:00.581721
go from [12.43383887 17.99196027] to [13.6        17.89500027], action vy++
GG 0 days 00:00:00.000213
routing 4.269481111111111
learning 3.361877
acting 0.09823197374701668
node_adding 0.2860065
A0 0 days 00:00:00.008341
GG 0 days 00:00:00.000142
routing 4.269481111111111
learning 3.361877
acting 0.09800144999999999
node_adding 0.2860065
A0 0 days 00:00:00.015634
go from [12.03383887 17.99196027] to [13.6        17.89500027], action vy++
GG 0 days 00:00:00.000197
routing 4.269481111111111
learning 3.361877
acting 0.0978471591448931
node_adding 0.2860065
A0 0 days 00:00:00.008066
go from [12.03383887 17.99196027] to [13.6        17.89500027], action vy++
GG 0 days 00:00:00.0001

go from [12.02372687 17.99196027] to [12.03383887 17.99196027], action vx--
GG 0 days 00:00:00.000115
routing 4.155444916666666
learning 3.12726575
acting 0.1202669054054054
node_adding 0.2860065
A0 0 days 00:00:00.006808
GG 0 days 00:00:00.000122
DistanceMeasure rmse 83.6656387961785 rmse/mean =  0.843906338171091 rmse/std =  1.343562229312927 shape 1391
ssr rmse 0.5759508934180259 ssr rmse/std 1.2732801837591845 ssr rmse/mean 1.617851870682894 s.shape (445, 3)
ssar 0.9970842309747034  rmse/std 1.7825574492795822 ssar rmse/mean 2.8008196649154664
routing 4.155444916666666
learning 4.5366293846153845
acting 0.11999944943820225
node_adding 0.2860065
A0 0 days 00:00:21.456138
go from [12.02372687 17.99196027] to [12.03383887 17.99196027], action vx--
GG 0 days 00:00:00.000099
routing 4.155444916666666
learning 4.5366293846153845
acting 0.11980657847533632
node_adding 0.2860065
A0 0 days 00:00:00.005470
predicting distances 2020-07-26 19:22:49.400754
edges_coord_np [100.   0.   0.]
edges_

In [ ]:
agent.debug_info(6000,1000)

In [ ]:
#agent.graph.edges(data=True)

In [ ]:
edges_values = agent.rewrite_edges()

In [ ]:
route = agent.make_route_to_all_states(state,agent.goal_state_list)
print(route)
agent.route_in_graph = route